In [1]:
import numpy as np
from bigfile import BigFile
import struct
import glob
import os, sys

In [2]:
class BHType:
    def __init__(self, name_sel=None):
        self.name_all = ('size','BHID','BHMass','Mdot','Density','timebin','Encounter','MinPos',\
             'MinPot','Entropy','GasVel','acMom','acMass','acBHMass',\
             'Fdbk','SPHID','SwallowID','CountProgs','Swallowed',\
             'BHpos','srDensity','srParticles','srVel','srDisp',\
             'DFAccel','DragAccel','GravAccel','BHvel','Mtrack','Mdyn',\
             'KineticFdbkEnergy','NumDM','V1sumDM','V2sumDM','MgasEnc','KEflag','z','size2')
        self.dtype_all = ('i','q','d','d','d','i','i','3d',\
            'd','d','3d','3d','d','d',\
            'd','q','q','i','i',\
            '3d','d','d','3d','d',\
            '3d','3d','3d','3d','d','d',\
            'd','d','3d','d','d','i','d','i')

        if name_sel is None:
            name_sel = self.name_all
        self.name_sel = name_sel
        self.name2type = {name: dtype for name, dtype in zip(self.name_all, self.dtype_all)}


    @property
    def TypeAll(self):
        np_type = np.dtype({'names':self.name_all, 'formats':self.dtype_all})
        return np_type
    @property
    def TypeSel(self):
        name_sel = [name for name in self.name_all if name in name_sel]
        dtype_sel = [dtype for name, dtype in zip(self.name_all, self.dtype_all) if name in name_sel]
        np_type = np.dtype({'names':name_sel, 'formats':dtype_sel})
        return np_type

In [8]:
np.seterr(all='raise')
def read_binary_file(file):
    dt = BHTYPE.TypeAll
    dtype = BHTYPE.dtype_all
    #-----------------------------------------------------------
    struct_fmt = ''.join(list(dtype))
    struct_len = struct.calcsize(struct_fmt)
    data = np.zeros(1, dtype=BHTYPE.TypeAll)
    nsingle = data.nbytes
    
    print(nsingle)
    print(struct_len)
    
    results = np.fromfile(file, dtype=dt, count=-1, offset=0)
    hastrouble = 0
    
    mask =  (results["size"] != nsingle - 8) | (results["size2"] != nsingle - 8)
    mask = mask.nonzero()[0]
    if len(mask > 0):
        hastrouble = 1
        istart = mask[0]
        ntrouble = sum(mask)
        print(f"starting of trouble {istart}, total troubles {ntrouble}")
        print(results[istart - 1])
        # print("================================")
        # for dd in BHTYPE.name_all:
        #     print(dd, results[istart][dd])

        offset = istart * nsingle
        print("=============start of bad data=============")
        trouble_data = np.fromfile(file, dtype=dt, count=1, offset=offset)
        for dd in ["size", "size2"]:
            print(dd, trouble_data[dd][0])
            
        # search for restart point
        for newoff in range(offset + 4, offset + nsingle):
            size1 = np.fromfile(file, dtype='i', count=1, offset=newoff)
            size2 = np.fromfile(file, dtype='i', count=1, offset=newoff+nsingle-4)
            if (size1 == nsingle - 8) & (size2 == nsingle - 8):
                print(f"resetting offset from {offset} to {newoff}")
                break
        print("=============good data=============")
        next_data = np.fromfile(file, dtype=dt, count=1, offset=newoff)
        for dd in ["size", "size2"]:
            print(dd, next_data[dd][0])

    
    return results, hastrouble

In [9]:


SEL_COL=['acBHMass', 'BHMass', 'Density', 'KEflag',  'Mdyn', 'NumDM', 'V1sumDM',
'acMass', 'BHpos', 'Encounter', 'KineticFdbkEnergy','MgasEnc', 'Swallowed', 'V2sumDM',
'BHID', 'BHvel', 'Entropy', 'DFAccel','DragAccel','GravAccel', 'Mdot', 'Mtrack', 'SwallowID',  'z']

BHTYPE = BHType(SEL_COL)

fflist = sorted(glob.glob("/home1/08942/nianyic/ASTRID2_PIG/BlackholeDetails-R599/*"))




ff = "/home1/08942/nianyic/ASTRID2_PIG/BlackholeDetails-R599/000AA4"
data, trouble = read_binary_file(ff)
    
# trouble_counter = 0
# for i in range(10):
#     print("----- %d --------"%i)
    
#     data, trouble = read_binary_file(fflist[i])
#     trouble_counter += trouble
# print(trouble_counter)

436
444
starting of trouble 80229, total troubles 44971139919
(428, 324876571846, 4.29174823e-05, 2.00740777e-08, 1.11898499e-05, 35, 0, [237137.89509666, 181845.94357425, 155723.65035466], -5068068.52494102, 17318216.47865558, [-45.02842866, 112.16108171, 133.01050517], [0., 0., 0.], 0., 0., 1.11898499e-05, 0, -1, 0, 0, [237144.68107342, 181847.51869297, 155721.013679  ], 0.00024702, 0., [-56.76425141, 116.95282117, 132.81412808], 231.96620254, [-0.24092741, -0.71444688,  0.10088284], [-6.99908393e-05, -6.62930196e-04,  8.80850536e-05], [-1328.26557947,   679.35853228,  -903.8279913 ], [-38.9430376 , 169.79990314, 125.35190295], 0.00012769, 0.0009958, 0., 0., [0., 0., 0.], 81.87653459, 0.08428787, 0, 0.57054673, 428)
=============start of bad data=============
size 1389784087
size2 428
resetting offset from 34979844 to 34980276
=============good data=============
size 428
size2 428
